# Concatenating, Appending, Joining DataFrames

This is a complex topic.  The examples below just demonstrate the basics -- you will need to do more investigation and/or experimentation when you have real work to do.

In [1]:
import numpy as np
import pandas as pd
np.__version__, pd.__version__

('1.14.3', '0.23.0')

In [2]:
A = pd.DataFrame({"A":[1, 2, 3], "B":[4, 5, 6]})
A

,A,B
0,1,4
1,2,5
2,3,6


In [3]:
B = pd.DataFrame({"C":[1, 2, 3], "D":[4, 5, 6]})
B

,C,D
0,1,4
1,2,5
2,3,6


In [12]:
# Careful ... indices don't have to be unique -- confusing.
pd.concat([A, B],sort=True)

,A,B,C,D
0,1.0,4.0,NaN,NaN
1,2.0,5.0,NaN,NaN
2,3.0,6.0,NaN,NaN
0,NaN,NaN,1.0,4.0
1,NaN,NaN,2.0,5.0
2,NaN,NaN,3.0,6.0


In [13]:
# try extracting row '1' - using the dictionary-type method
pd.concat([A,B],sort=True).loc[1]

,A,B,C,D
1,2.0,5.0,NaN,NaN
1,NaN,NaN,2.0,5.0


In [14]:
# or row '1' - using Python implicit numbering style
pd.concat([A,B],sort=True).iloc[1]

A    2.0
B    5.0
C    NaN
D    NaN
Name: 1, dtype: float64

In [15]:
# Catching repeats
# We didn't go over exception handling -- look up 'try/catch' for details :-)
try:
    pd.concat([A, B], verify_integrity=True,sort=True)
except ValueError as e:
    print("ValueError:", e)

ValueError: Indexes have overlapping values: Int64Index([0, 1, 2], dtype='int64')


In [17]:
# if indexes don't matter -- ignore them and create a new
# implicit index with the combined dataframe
pd.concat([A, B], ignore_index=True,sort=True)

,A,B,C,D
0,1.0,4.0,NaN,NaN
1,2.0,5.0,NaN,NaN
2,3.0,6.0,NaN,NaN
3,NaN,NaN,1.0,4.0
4,NaN,NaN,2.0,5.0
5,NaN,NaN,3.0,6.0


In [18]:
# specify that the concatenation should happen along axis 1
pd.concat([A, B], axis=1)

,A,B,C,D
0,1,4,1,4
1,2,5,2,5
2,3,6,3,6


In [19]:
C = pd.DataFrame({"A":[7, 8, 9], "B":[10, 11, 12]})
C

,A,B
0,7,10
1,8,11
2,9,12


In [20]:
pd.concat([A,C])

,A,B
0,1,4
1,2,5
2,3,6
0,7,10
1,8,11
2,9,12


In [21]:
pd.concat([A,C], ignore_index=True)

,A,B
0,1,4
1,2,5
2,3,6
3,7,10
4,8,11
5,9,12


In [22]:
pd.concat([A,C], axis=1)

,A,B,A,B
0,1,4,7,10
1,2,5,8,11
2,3,6,9,12


In [23]:
pd.concat([A,C], axis=1)['B']

,B,B
0,4,10
1,5,11
2,6,12


In [24]:
pd.concat([A,C], axis=1, ignore_index=True)

,0,1,2,3
0,1,4,7,10
1,2,5,8,11
2,3,6,9,12


In [25]:
D = pd.DataFrame({"C":[7, 8, 9], "D":[10, 11, 12]}, index=['x', 'y', 'z'])
D

,C,D
x,7,10
y,8,11
z,9,12


In [26]:
E = pd.DataFrame({"A":[7, 8, 9], "D":[10, 11, 12]}, index=['x', 'y', 'z'])
E

,A,D
x,7,10
y,8,11
z,9,12


In [27]:
pd.concat([A,D], axis=1)

,A,B,C,D
0,1.0,4.0,NaN,NaN
1,2.0,5.0,NaN,NaN
2,3.0,6.0,NaN,NaN
x,NaN,NaN,7.0,10.0
y,NaN,NaN,8.0,11.0
z,NaN,NaN,9.0,12.0


In [30]:
# inner join involves including items that occur in both
pd.concat([A,D], axis=1, join="inner")

,A,B,C,D


In [29]:
pd.concat([A,D], axis=1, join="outer")

,A,B,C,D
0,1.0,4.0,NaN,NaN
1,2.0,5.0,NaN,NaN
2,3.0,6.0,NaN,NaN
x,NaN,NaN,7.0,10.0
y,NaN,NaN,8.0,11.0
z,NaN,NaN,9.0,12.0


In [31]:
pd.concat([A, E], join_axes=[A.columns])

,A,B
0,1,4.0
1,2,5.0
2,3,6.0
x,7,NaN
y,8,NaN
z,9,NaN


In [32]:
A

,A,B
0,1,4
1,2,5
2,3,6


In [33]:
E

,A,D
x,7,10
y,8,11
z,9,12


## Joins

If you have a database background, you will recognize this topic.  If not, we'll do a quick run-through here and will then circle back later in the semester when we get to the MySQL component.

In [34]:
# Assume that we have a dataframe with a list of orders.
orders = pd.DataFrame({
          'order' : [123, 456, 789, 823, 950, 1024], 
            'sku' : ['A109', 'A227', 'A876', 'A109', 'A227', 'B552'], 
    'sales_price' : [765.55, 227.83, 23.50, 745.87, 235.25, 86.50]})
orders

,order,sku,sales_price
0,123,A109,765.55
1,456,A227,227.83
2,789,A876,23.50
3,823,A109,745.87
4,950,A227,235.25
5,1024,B552,86.50


In [35]:
# And we have a separate dataframe with the sku information.
skus = pd.DataFrame({
     'sku' : ['A100', 'A109', 'A200', 'A227', 'A300', 'A876', 'A904'],
    'name' : ['Widget1', 'Widget2', 'Widget3', 'Widget4', 'Widget5', 'Widget6', 'Widget7'],
    'cost' : [12.50, 423.50, 96.50, 86.34, 1850.45, 3.23, 7.50]
})
skus

,sku,name,cost
0,A100,Widget1,12.50
1,A109,Widget2,423.50
2,A200,Widget3,96.50
3,A227,Widget4,86.34
4,A300,Widget5,1850.45
5,A876,Widget6,3.23
6,A904,Widget7,7.50


In [42]:
# We'd like to join (merge) the data so that the sku information can be 
# easily combined with the order information.
# sales = pd.merge(orders, skus, how="right")
#sales = pd.merge(orders, skus, how="left")
sales = pd.merge(orders, skus)
sales

,order,sku,sales_price,name,cost
0,123,A109,765.55,Widget2,423.50
1,823,A109,745.87,Widget2,423.50
2,456,A227,227.83,Widget4,86.34
3,950,A227,235.25,Widget4,86.34
4,789,A876,23.50,Widget6,3.23


In [37]:
# now that we have the data we need, let's add a calculated column
sales['profit'] = sales['sales_price'] - sales['cost']
sales

,order,sku,sales_price,name,cost,profit
0,123,A109,765.55,Widget2,423.50,342.05
1,823,A109,745.87,Widget2,423.50,322.37
2,456,A227,227.83,Widget4,86.34,141.49
3,950,A227,235.25,Widget4,86.34,148.91
4,789,A876,23.50,Widget6,3.23,20.27


In [43]:
# Notice that our previous merge did not include
# all of the SKU or Orders.
# Join types: inner, left, right, outer
sales = pd.merge(orders, skus, how="inner")
sales

,order,sku,sales_price,name,cost
0,123,A109,765.55,Widget2,423.50
1,823,A109,745.87,Widget2,423.50
2,456,A227,227.83,Widget4,86.34
3,950,A227,235.25,Widget4,86.34
4,789,A876,23.50,Widget6,3.23
